In [1]:
import json
import numpy as np
import yaml
config = 'Antenna_confi/FYST_config'
from LaserMetrology import Antenna
import matplotlib.pyplot as plt

from LaserMetrology.coordinate import coord_sys,global_coord
Meas_coord = coord_sys([0,0,0],
                       angle = [0,0,0],
                       axes='xyz',
                       ref_coord=global_coord)

In [2]:
FYST = Antenna()

In [3]:
panel = FYST.M2.panels['12281']
#z0 = FYST.M2.surface.surface(panel.center[0], panel.center[1]) -30
q = 300
x,y =np.meshgrid(np.linspace(-q,q,2),np.linspace(-q,q,2))
x = np.append(x.T.ravel(),0) + np.random.normal(0,5,5)
y = np.append(np.flip(y).T.ravel(),0) + np.random.normal(0,5,5)

x_0 = x+panel.center[0]
y_0 = y+panel.center[1]
dr= 0.01 - 0.05*x/q +0.05*y/q -0.009*x*y/q**2 + 0.03*((x/q)**2+(y/q)**2)
z_0 = panel.surface.surface(x_0,y_0)
nx,ny,nz,N = panel.surface.normal_vector(x_0,y_0)
x_0 = x_0 - dr*nx
y_0 = y_0 - dr*ny
z_0 = z_0 - dr*nz

offset = 100# mm
x_meas,y_meas,z_meas = x_0 - offset*nx, y_0 - offset*ny, z_0 - offset*nz,

Meas_data = np.column_stack([x_meas,y_meas,z_meas])
print(Meas_data)
#x_g,y_g,z_g = Meas_coord.To_coord_sys(global_coord,x_meas,y_meas,z_meas)
#Meas_data_Laser = np.column_stack([x_g,y_g,z_g])
Meas_data_Laser = np.column_stack([FYST.M2.coord_sys.To_coord_sys(Meas_coord,x_meas,y_meas,z_meas)]).T
print(Meas_data_Laser)

[[ 1787.23752058 -1824.96044484   178.46623732]
 [ 1802.26288749 -2426.37900718   207.82046693]
 [ 2399.25585253 -1834.80691378   212.07072769]
 [ 2399.66796239 -2428.53295703   240.69379476]
 [ 2102.72085632 -2124.68474179   207.51977269]]
[[ 1787.23752058 -5207.79522593  1787.74547316]
 [ 1802.26288749 -5370.13260698  2367.58384084]
 [ 2399.25585253 -5179.02893405  1807.7133267 ]
 [ 2399.66796239 -5339.62736867  2380.02271611]
 [ 2102.72085632 -5275.01376702  2081.27644228]]


In [4]:
x_meas

array([1787.23752058, 1802.26288749, 2399.25585253, 2399.66796239,
       2102.72085632])

In [5]:
Meas_coord.To_coord_sys(global_coord,x_meas,y_meas,z_meas)

(array([1787.23752058, 1802.26288749, 2399.25585253, 2399.66796239,
        2102.72085632]),
 array([-1824.96044484, -2426.37900718, -1834.80691378, -2428.53295703,
        -2124.68474179]),
 array([178.46623732, 207.82046693, 212.07072769, 240.69379476,
        207.51977269]))

In [6]:
a,b,c,d,e = panel.To_adjuster_position_Method1(Meas_data,offset)
FYST.Offset_surface(offset)
a1,b1,c1,d1,e1 = panel.To_adjuster_position_Method2(Meas_data)

110.11527560381121 35.20488699057708 35.32025237807127 -23.19509831132624 9.998317201166028
v1: 107.58134477436339um
v2: 32.787881230391335um
v3: 32.77213009052993um
v4: -26.006252382602945um
v5: 9.999723032432483um


In [7]:
p = 200
x,y =np.meshgrid(np.linspace(-p,p,2),np.linspace(-p,p,2))
x = np.append(x.T.ravel(),0)
y = np.append(np.flip(y).T.ravel(),0)
dz_meas= a + b*x +c*y +d*x*y + e*((x)**2+(y)**2)
print(dz_meas*1000)
dz_meas1= a1 + b1*x +c1*y +d1*x*y + e1*((x)**2+(y)**2)
print(dz_meas1*1000)


[110.1152756   35.20488699  35.32025238 -23.19509831   9.9983172 ]
[107.58134477  32.78788123  32.77213009 -26.00625238   9.99972303]


In [8]:
dz= 0.01 - 0.05*x/q +0.05*y/q -0.009*x*y/q**2 + 0.03*((x/q)**2+(y/q)**2)
print(dz*1000)
print((dz_meas - dz)*1000)
print((dz_meas1 - dz)*1000)

[107.33333333  32.66666667  32.66666667 -26.          10.        ]
[ 2.78194227e+00  2.53822032e+00  2.65358571e+00  2.80490169e+00
 -1.68279883e-03]
[ 0.24801144  0.12121456  0.10546342 -0.00625238 -0.00027697]


In [9]:
FYST.M1.panels['11531'].center

[1342.4, 35.0]

In [10]:
FYST.M2.panels.keys()

dict_keys(['12391', '12491', '12591', '12691', '12791', '12281', '12381', '12481', '12581', '12681', '12781', '12881', '12171', '12271', '12371', '12471', '12571', '12671', '12771', '12871', '12971', '12161', '12261', '12361', '12461', '12561', '12661', '12761', '12861', '12961', '12151', '12251', '12351', '12451', '12551', '12651', '12751', '12851', '12951', '12141', '12241', '12341', '12441', '12541', '12641', '12741', '12841', '12941', '12131', '12231', '12331', '12431', '12531', '12631', '12731', '12831', '12931', '12221', '12321', '12421', '12521', '12621', '12721', '12821', '12311', '12411', '12511', '12611', '12711'])

In [11]:
import json
import numpy as np
import yaml
config = 'Antenna_confi/FYST_config'
from LaserMetrology import Antenna
import matplotlib.pyplot as plt

In [12]:
FYST = Antenna()

In [13]:
FYST.M2.surface._create_offset_surface(10)

In [14]:
FYST.M1.surface._create_offset_surface(-10)

In [15]:
FYST.M1.panels['11531'].surface.surface_offset(0,0)

array([[-10.]])